# Projeto 1 - Ciência dos Dados

Nome: Arthur Gomes Chieppe

Nome: João Pedro Di Jardim Marquez

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'PS5.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo PS5.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [12]:
train = pd.read_excel(filename)
train.head(20)

,Treinamento,Relevância:
0,@immandy4 o arquiteto do ps5 disse que ele ser...,1
1,o vídeo que eu fiz pro evento tá no youtube tb...,0
2,@chief117_com_br @vingadorbrambz enquanto o da...,1
3,possivelmente hoje sai o valor e a data de lan...,1
4,"@lealemy_ sim, mas os produtos da sony vão par...",1
5,"não oficial, este modelo preto para o #ps5 ser...",1
6,@marcelo_gj1 @omattmeyer @lv_ogandojr @marccon...,1
7,hoje as 17:00 finalmente saberemos a data e pr...,1
8,"falta pouco, hein! hj às 17h promete! #playsta...",0
9,só falta o ps5 agr 😭 https://t.co/bwgw1hee77,0


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância:
0,"sony nega, mas ações continuam em queda após r...",1.0
1,hoje tem \n\n#ps5,0.0
2,@prodigyrr não faça isso. ps5 ta uma completa ...,0.0
3,"hoje lançam o preço do ps5, tô só por saber qu...",1.0
4,"@1playergirl bom, nesse momento me sinto num h...",0.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto trata-se do console de videogames PlayStation5. Quando extraímos os Tweets, havia uma espera coletiva acerca da livestream de lançamento do novo PlayStation 5. Assim , consideramos relevante especulações acerca do preço do novo console, bem como opiniões dos usuários sobre seu design. Ademais, consideramos algumas comparações entre o novo PlayStation e Xbox como relevantes, porém somente aquelas bem fundamentadas e intencionadas, já que consideramos irrelevantes comparações vagas e com discurso de ódio. Anúncios feitos acerca da live também foram consideradas como relevantes, com exceção de anúncios que tinham como objetivo somente a autopromoção dos usuários.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [37]:
#Limpando caracteres:
train_limpo = train.iloc[:, 0]
ls= []
for w in train_limpo:
    w.replace('a','')
    w.replace(":",'')
    w.replace('"','')
    w.replace("'",'')
    w.replace("(",'')
    w.replace(")",'')
    w.replace("\n",'')
    ls.append(w)
    
    

In [35]:
train_limpo.head(20)
print(ls)

['@immandy4 o arquiteto do ps5 disse que ele será muito silêncioso e terá um sistema de resfriamento robusto', 'o vídeo que eu fiz pro evento tá no youtube tbm!\n\n#ps5 #ps5showcase #playfivebrasil \n\nhttps://t.co/wcyyfxzrke', '@chief117_com_br @vingadorbrambz enquanto o datacenter da ms tá funcionando que é uma beleza no fundo do oceano, a sony tendo problema na fabricação do ps5 não vai ter sopro que faça ela navegar nesse mar.', 'possivelmente hoje sai o valor e a data de lançamento do ps5. vamos ver qual vai ser a estratégia da sony, depois da bomba que a microsoft jogou no mercado. https://t.co/kgeddwo0f6', '@lealemy_ sim, mas os produtos da sony vão parar de ser vendidos no brasil. no caso do ps5, ele vai ser por distribuidores parceiros o que significa que o produto vai custar mais caro do que se esperava.', 'não oficial, este modelo preto para o #ps5 seria uma boa pedida. melhor do que o original? https://t.co/9jq5jhypy6', '@marcelo_gj1 @omattmeyer @lv_ogandojr @marcconserv @t

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**